In [ ]:
!pip install torch transformers accelerate


In [ ]:
import os
from huggingface_hub import login
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

#### Login to Hugging Face

In [ ]:
#Login to hugging face

# Replace 'your_huggingface_token' with your actual token
token=os.environ["HF_TOKEN"]
login(token)

In [ ]:
# We will use the default dataset of the project to download the model. But it can be any dataset you have access to
def get_download_dataset_folder(ds_name,model_name):
    ds_dir = os.environ['DOMINO_DATASETS_DIR']    
    download_ds_dir = f"{ds_dir}/{ds_name}/{model_name}"
    return download_ds_dir

In [ ]:
# Save the model to a dataset
def download_model(model_name,save_path):
    print(f"Downloading {model_name} to location {save_path}")
    # Load the model and tokenizer from Hugging Face
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Save them locally
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"Model saved locally at {save_path}")

#### Make sure you accept the terms of service for the model you are downloading from HF

1. Download the `Mistral-7B-Instruct-v0.3` model
2. Download the `google/gemma-2b` model

In [ ]:

# Choose the model (base or instruct)
#model_name = "mistralai/Mistral-7B-Instruct-v0.3" 
#save_path = f"{download_ds_dir}/{model_name}"  # Set the directory to save the model

#download_model(model_name,save_path)

model_name = "google/gemma-2b"
ds_name = "llmstore"
save_path = get_download_dataset_folder(ds_name,model_name)
download_model(model_name,save_path)


### Test the downloaded model locally

In [ ]:
model_name = "google/gemma-2b"
ds_name = "llmstore"
model_path = get_download_dataset_folder(ds_name,model_name)

model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text-generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time in a distant land,"
output = text_generator(prompt, max_length=50, do_sample=True)
print(output[0]['generated_text'])

